In [94]:
using Plots

┌ Info: Recompiling stale cache file /home/etienne/.julia/compiled/v1.2/Plots/ld3vC.ji for Plots [91a5bcdd-55d7-5caf-9e0b-520d859cae80]
└ @ Base loading.jl:1240


In [1]:
using LinearAlgebra

# 1D Finite volume for heat conduction

Constant coefficients and no heat generation inside the material, the Fourier equation:
$$
\rho C_p \, \frac{\partial T}{\partial t} = -k \, \frac{\partial^2 T}{\partial x^2}
$$


http://www.csc.kth.se/utbildning/kth/kurser/DN2255/ndiff13/Lecture3.pdf


$$
\Delta x \sim \sqrt{ \alpha \Delta t }
$$


$$
\rho C_p dx \, \frac{\partial T_i}{\partial t} = -k \, \frac{T_{i-1} - 2T_{i} + T_{i+1}}{dx}
$$


Boundary coundary conditions:

$$
\rho C_p dx \, \frac{\partial T_1}{\partial t} = -k \, \frac{- T_1 + T_{2}}{dx} + S_1(T,\, t)
$$

Considering convection BC: $S_1(T, t) = h_1\, \left( T_{source} - T_1 \right)$



$$
\rho C_p dx \, \frac{\partial T_1}{\partial t} =
-k \, \frac{(-1 + h_1dx/k)T_1 + T_{2}}{dx} + h_1\,  T_{source} 
$$

fixed temperature with $h_1 = k/dx$ and $T_{source} = T_{BC}$

Linear system (matrix) formulation:
$$
\frac{\partial T}{\partial t} = A \, T + S(t)
$$

Time discretization using theta-method:
$$
T^{k+1} - T^{k} = \theta\, dt \left[ A \, T^{k+1} + S^{k+1} \right]
+ (1-\theta)\, dt \left[ A \, T^{k} + S^{k} \right]
$$

$$
 \left[ I  - \theta\, dt\, A \right] T^{k+1} =
 \theta\, dt \, S^{k+1} 
+ (1-\theta)\, dt \left[ A \, T^{k} + S^{k} \right] + T^k
$$

$$
\left[ I  - \theta\, dt\, A \right] T^{k+1} =
\left[I + (1-\theta)\, dt \,A \right] T^{k}
+ \left( \theta \, S^{k+1} +  (1-\theta)\, S^{k}  \right) dt
$$


In [4]:
struct Material
    """
    thermal conductivity
    density
    specific heat
    Thermal diffusivity
    """
    k
    rho
    Cp
    alpha
end

Material(k, rho, Cp) = Material(k, rho, Cp, k/(rho*Cp))

Material

In [5]:
# -- some units --
min = 60.0 # seconds
cm = 0.01   # meters

0.01

In [6]:
# -- material properties --
mat = Material(0.04, 160.0, 2100.0)

# 
dt = 5min
thickness = 20cm

# buidl a wall
delta_x = sqrt( dt*mat.alpha )
println(delta_x)

N = Int(ceil( thickness/delta_x ))

dx = thickness / N

0.005976143046671968


0.0058823529411764705

In [7]:
# -- Build the matrix --
M = SymTridiagonal(-2.0 * ones(N), ones(N-1));
M[1, 1] = -1.0     #- h_left*dx/k
M[end, end] = -1.0 #- h_right*dx/k

A = mat.alpha / dx^2 * M;

In [21]:
M

34×34 SymTridiagonal{Float64,Array{Float64,1}}:
 -1.0   1.0    ⋅     ⋅     ⋅     ⋅   …    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
  1.0  -2.0   1.0    ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅    1.0  -2.0   1.0    ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅    1.0  -2.0   1.0    ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅    1.0  -2.0   1.0       ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅    1.0  -2.0  …    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅    1.0       ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅   …    ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅     ⋅     ⋅     ⋅     ⋅ 
   ⋅     ⋅     ⋅     ⋅     ⋅     ⋅        ⋅     ⋅   

In [20]:
M

# -- Boundary conditions: Convection --
h_left = 10.0
h_right = .0

period = 30000
w = 2*pi/period
T_ext(t) = 1. * sin(w*t)
T_int(t) = 0.0

T_int (generic function with 1 method)

In [8]:
dx = L/N
println(dx)

0.002


In [9]:
dt_min = dx^2 /( 2*kappa )
println("dt_min ", dt_min)

dt = dt_min

dt_min 16.8


16.8

In [10]:
# -- Build the matrix --
M = SymTridiagonal(-2.0 * ones(N), ones(N-1));
M[1, 1] = -1.0 - h_left*dx/k
M[end, end] = -1.0 - h_right*dx/k

A = k/(rho*Cp*dx^2) * M;

In [ ]:
inv_A = factorize( I - dt*theta*A );

In [11]:
function update_S!(S, t)
    fill!(S, 0.0)
    S[1]   = h_left*T_ext(t)/(rho*Cp*dx)
    S[end] = h_right*T_int(t)/(rho*Cp*dx)
end 

update_S! (generic function with 1 method)

In [16]:
t = 0
T = zeros(N);
S = zeros(N);
update_S!(S, t)

0.0

0.0

In [24]:
typeof((I + (1- 0.5 )*dt*A))

SymTridiagonal{Float64,Array{Float64,1}}

In [19]:
mutable struct LinearSystem
    A::SymTridiagonal{Float64,Array{Float64,1}}
    update_S!::Function
    T::Array{Float64,1}
    dt::Float64
    t::Float64
    theta::Float64
    I_minus_A::LDLt{Float64,SymTridiagonal{Float64,Array{Float64,1}}}
    I_plus_A::SymTridiagonal{Float64,Array{Float64,1}}
    St::Array{Float64,1}
end

In [20]:
LinearSystem(A, S0, T0, dt, update_S!) = LinearSystem(
    A,
    update_S!,
    T0,
    dt,
    0.0, 0.5,
    factorize( I - dt* 0.5 *A ),
    (I + (1 - 0.5)*dt*A),
    S0)

LinearSystem

In [22]:
state = LinearSystem(A, S, ones(size(A)[1]), 10, update_S!)

LinearSystem([-0.04464285714285715 0.029761904761904767 … 0.0 0.0; 0.029761904761904767 -0.059523809523809534 … 0.0 0.0; … ; 0.0 0.0 … -0.059523809523809534 0.029761904761904767; 0.0 0.0 … 0.029761904761904767 -0.029761904761904767], update_S!, [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0  …  1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0], 10.0, 0.0, 0.5, LDLt{Float64,SymTridiagonal{Float64,Array{Float64,1}}}([1.2232142857142858 -0.12165450121654503 … 0.0 0.0; -0.12165450121654503 1.2795156992237284 … 0.0 0.0; … ; 0.0 0.0 … 1.2803232004702307 -0.11622809284004992; 0.0 0.0 … -0.11622809284004992 1.1315136766607068]), [0.7767857142857142 0.14880952380952384 … 0.0 0.0; 0.14880952380952384 0.7023809523809523 … 0.0 0.0; … ; 0.0 0.0 … 0.7023809523809523 0.14880952380952384; 0.0 0.0 … 0.14880952380952384 0.8511904761904762], [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0])

In [36]:
function iter!(state)
    theta = state.theta
    dt = state.dt
    A = state.A
    
    state.t += dt
    old_S = state.St
    state.update_S!(state.St, state.t)
    S_theta = theta*state.St + (1 - theta)*old_S

    state.T = state.I_minus_A \ ( state.I_plus_A*T + dt*S_theta )
    
    return state.t, state.T
end

iter! (generic function with 1 method)

In [32]:
iter!(state)

(1.161006e7, [0.0015506408653885207, 0.00018022803046395237, 2.0947560257143604e-5, 2.4346949783398266e-6, 2.829799539796847e-7, 3.289022036302386e-8, 3.822767585883237e-9, 4.443129858779704e-10, 5.164165097266453e-11, 6.0022106036643076e-12  …  1.1708769898735872e-87, 1.360887994833055e-88, 1.581734162083672e-89, 1.8384194503906772e-90, 2.1367598656998347e-91, 2.4835152499578405e-92, 2.88654322634021e-93, 3.3550443410822416e-94, 3.9055439083504585e-95, 5.058994699935829e-96])

In [33]:
for i in 1:87600
    S_theta = theta*S(t+dt) + (1-theta)*S(t)
    T_iplus = inv_A\( *T + dt*S_theta )
    t = t + dt
    T[:] = T_iplus[:]
end
println(t)

plot(T)

LoadError: syntax: "*" is not a unary operator

In [28]:
using BenchmarkTools

In [35]:
@btime iter!(state)
#   4.215 μs (29 allocations: 10.81 KiB)
#   4.050 μs (27 allocations: 10.78 KiB)
#   3.890 μs (29 allocations: 10.81 KiB)  with type def
#   3.370 μs (25 allocations: 9.89 KiB)   with in place S
#   2.633 μs (19 allocations: 6.33 KiB)  with I_plus_A

  2.646 μs (19 allocations: 6.33 KiB)


(2.252011e7, [-0.10825992512344006, -0.012582844628104044, -0.0014624800336272004, -0.00016998126512514155, -1.9756598264034096e-5, -2.296271737235724e-6, -2.6689128466141645e-7, -3.1020265011827304e-8, -3.605426241717616e-9, -4.190518159503073e-10  …  -8.174623672175255e-86, -9.501209191020556e-87, -1.1043074239466832e-87, -1.283515457945209e-88, -1.4918055381539028e-89, -1.7338971324994161e-90, -2.0152761385587893e-91, -2.3423660323848003e-92, -2.726704168075626e-93, -3.5320002177145423e-94])

In [50]:
function test(state, a)
    state.I_plus_A * a
end

test (generic function with 1 method)

In [56]:
typeof(S[1])

Float64

In [55]:
@btime test(state, a) setup=(a=rand(N))
#   1.063 μs (1 allocation: 896 bytes)    inverse
#   175.217 ns (1 allocation: 896 bytes)  sum
#   209.491 ns (1 allocation: 896 bytes) produit
# #   465.347 ns (11 allocations: 176 bytes)  call update_S

  211.202 ns (1 allocation: 896 bytes)


100-element Array{Float64,1}:
 0.3083886413796859 
 0.08095377048229922
 0.14431180561960052
 0.7183941628590877 
 0.7524611105788032 
 0.4909881156404826 
 0.19572680339006504
 0.29887391782591705
 0.5349496612734986 
 0.7200684541103309 
 0.7045988496623894 
 0.3981357217078018 
 0.8149949443272301 
 ⋮                  
 0.4958409467444562 
 0.6600161657419276 
 0.6906672687447506 
 0.6381906421804872 
 0.23087101213649083
 0.24219464010839561
 0.5085407009324917 
 0.46760894196259134
 0.2629401558558435 
 0.7827942116991566 
 0.601211058237685  
 0.65648870926009   

In [54]:
@btime state.update_S!(state.St, t) setup=(t=rand())
#   465.347 ns (11 allocations: 176 bytes)

  473.959 ns (11 allocations: 176 bytes)


0.0